In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import modin.pandas as pd
from fastai.tabular import *
from utils import isVaildDate, is_more, purge_pat_files
from tqdm import tqdm, trange

2019-05-09 16:28:06,117	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-09_16-28-06_14175/logs.
2019-05-09 16:28:06,223	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:57372 to respond...
2019-05-09 16:28:06,333	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:27260 to respond...
2019-05-09 16:28:06,335	INFO services.py:804 -- Starting Redis shard with 10.0 GB max memory.
2019-05-09 16:28:06,345	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-09_16-28-06_14175/logs.
2019-05-09 16:28:06,346	WARNING services.py:1304 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 33710702592 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--sh

## 数据清洗

In [3]:
root = Path('../test_A')
root
root.ls()

PosixPath('../test_A')

[PosixPath('../test_A/ad_static_feature.out'),
 PosixPath('../test_A/ad_operation.dat'),
 PosixPath('../test_A/.ipynb_checkpoints'),
 PosixPath('../test_A/imps_log'),
 PosixPath('../test_A/test_sample.dat'),
 PosixPath('../test_A/user')]

In [4]:
u_data = root/'user'
u_data.ls()
log_data = root/'imps_log'
log_data.ls()

[PosixPath('../test_A/user/user_data')]

[PosixPath('../test_A/imps_log/totalExposureLog.out')]

- 广告静态数据读入以备用

In [5]:
ad_static = pd.read_csv('../data/ad_static.csv', low_memory=False, encoding='utf-8')

In [6]:
ad_static.shape
ad_static.tail()

(502153, 7)

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
502148,82588,2019-03-19 03:56:04,21666,21917,18,198,64
502149,665036,2019-02-21 09:08:35,6184,11633,18,232,64
502150,491914,2019-01-25 05:01:10,1809,5278,5,232,64
502151,3990,2019-03-01 08:17:08,4255,3401,18,218,36
502152,222150,2019-02-28 07:56:53,4250,14527,18,94,36


### 历史曝光日志数据文件

In [7]:
col_names = ['广告请求id', '广告请求时间', '广告位id', '用户id', '曝光广告id', '曝光广告素材尺寸', '曝光广告出价bid',
             '曝光广告pctr', '曝光广告quality_ecpm', '曝光广告totalEcpm']

In [8]:
c_sz = 102400

In [9]:
logs = pd.read_csv(log_data/'totalExposureLog.out', sep='\t', header=None, names=col_names, chunksize=c_sz)

In [10]:
def invalid_date(df_row, field='广告请求时间'):
    """是否删除当前行,首先转为时间格式之后再行本操作"""
    if not isVaildDate(str(df_row[field])):
        df_row[field] = np.nan
    
    return df_row

- 看下文件信息

In [11]:
!du -h {log_data}/'totalExposureLog.out'

6.2G	../test_A/imps_log/totalExposureLog.out


In [12]:
# !cat {log_data}/'totalExposureLog.out' | wc -l

6.2G 数据 1亿多行数据

- 删除空值 数据量大 考虑删除而不是插值，因为我们不缺数据

In [13]:
102386695 / 102400

999.870068359375

- 获取有效广告 id

In [14]:
ad_id = ad_static['广告id'].unique()

In [15]:
ad_id.shape

(502153,)

In [16]:
def invalid_resquest_ad_id(df_row, field=None):
    if df_row[field] in ad_id:
        return df_row

- 删除非法多值

In [17]:
def invalid_more_value(df_row, field=None):
    if len(str(df_row[field]).split(',')) == 1:
        return df_row

- 首先应该删除 `'../data/ad_logs.csv'` 因为接下来我们是以 append 方式写文件

In [18]:
?partial

Init signature: partial(self, /, *args, **kwargs)
Docstring:     
partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.
File:           ~/Softwares/miniconda3/envs/fastai/lib/python3.6/functools.py
Type:           type
Subclasses:     


- 生成器 读写同时进行（备份）

In [19]:
logs = pd.read_csv(log_data/'totalExposureLog.out', sep='\t', header=None, names=col_names, iterator=True)

In [20]:
p=logs.get_chunk(50)

In [21]:
p.tail()

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm
45,58681445,1550366900,25,700816,537751,30,26,1.500,30.00,69.000
46,60671671,1550366351,39,1386990,78542,44,401,3.307,66.14,1392.140
47,47255352,1550353895,265,1082782,12217,1,120,0.891,17.82,124.740
48,58794695,1550418550,215,945136,256765,56,101,11.631,232.62,1407.351
49,64166811,1550394365,39,983464,301669,36,135,26.660,533.20,4132.200


In [22]:
p['广告请求时间'] = pd.to_datetime(p['广告请求时间'], unit='s')
p.head()

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm
0,53991770,2019-02-17 13:22:26,94,1160618,451525,50,46,47.217,944.34,3122.340
1,25942318,2019-02-17 02:34:52,79,203814,214797,64,10,49.094,981.88,1471.880
2,66156247,2019-02-17 15:16:40,18,808543,92253,40,96,3.824,76.48,443.584
3,5935886,2019-02-17 01:11:38,198,7270,160082,64,85,6.123,122.46,642.915
4,11624425,2019-02-16 23:52:39,168,852707,253902,64,60,4.329,86.58,346.320


In [23]:
p = p.apply(invalid_date, axis=1)
p.head()

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm
0,53991770,2019-02-17 13:22:26,94,1160618,451525,50,46,47.217,944.34,3122.340
1,25942318,2019-02-17 02:34:52,79,203814,214797,64,10,49.094,981.88,1471.880
2,66156247,2019-02-17 15:16:40,18,808543,92253,40,96,3.824,76.48,443.584
3,5935886,2019-02-17 01:11:38,198,7270,160082,64,85,6.123,122.46,642.915
4,11624425,2019-02-16 23:52:39,168,852707,253902,64,60,4.329,86.58,346.320


In [24]:
p['广告请求时间_date'] = p['广告请求时间'].apply(lambda x: x.date())
p.head()

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm,广告请求时间_date
0,53991770,2019-02-17 13:22:26,94,1160618,451525,50,46,47.217,944.34,3122.340,2019-02-17
1,25942318,2019-02-17 02:34:52,79,203814,214797,64,10,49.094,981.88,1471.880,2019-02-17
2,66156247,2019-02-17 15:16:40,18,808543,92253,40,96,3.824,76.48,443.584,2019-02-17
3,5935886,2019-02-17 01:11:38,198,7270,160082,64,85,6.123,122.46,642.915,2019-02-17
4,11624425,2019-02-16 23:52:39,168,852707,253902,64,60,4.329,86.58,346.320,2019-02-16


In [25]:
def save_csv(row):
    date = str(row['广告请求时间_date'])  
    pd.DataFrame(row).T.to_csv(f'../data/{date}_log.csv', mode='a', index=None, encoding='utf-8', header=False)

In [26]:
p = pd.merge(p, ad_static, left_on='曝光广告id', right_on='广告id', how='inner')
p.head()

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm,广告请求时间_date,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
0,53991770,2019-02-17 13:22:26,94,1160618,451525,50,46,47.217,944.34,3122.340,2019-02-17,451525,2019-02-15 10:11:02,25082,-1,2,172,50
1,25942318,2019-02-17 02:34:52,79,203814,214797,64,10,49.094,981.88,1471.880,2019-02-17,214797,2019-02-13 02:57:43,22770,342,18,232,64
2,66156247,2019-02-17 15:16:40,18,808543,92253,40,96,3.824,76.48,443.584,2019-02-17,92253,2018-10-18 07:10:57,29218,32847,13,172,40
3,5935886,2019-02-17 01:11:38,198,7270,160082,64,85,6.123,122.46,642.915,2019-02-17,160082,2019-02-16 08:29:28,12759,-1,1,90,64
4,11624425,2019-02-16 23:52:39,168,852707,253902,64,60,4.329,86.58,346.320,2019-02-16,253902,2019-02-15 08:27:12,5297,-1,1,230,64


In [27]:
p.dropna(axis=0, how='any', inplace=True)
p.head()

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm,广告请求时间_date,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
0,53991770,2019-02-17 13:22:26,94,1160618,451525,50,46,47.217,944.34,3122.340,2019-02-17,451525,2019-02-15 10:11:02,25082,-1,2,172,50
1,25942318,2019-02-17 02:34:52,79,203814,214797,64,10,49.094,981.88,1471.880,2019-02-17,214797,2019-02-13 02:57:43,22770,342,18,232,64
2,66156247,2019-02-17 15:16:40,18,808543,92253,40,96,3.824,76.48,443.584,2019-02-17,92253,2018-10-18 07:10:57,29218,32847,13,172,40
3,5935886,2019-02-17 01:11:38,198,7270,160082,64,85,6.123,122.46,642.915,2019-02-17,160082,2019-02-16 08:29:28,12759,-1,1,90,64
4,11624425,2019-02-16 23:52:39,168,852707,253902,64,60,4.329,86.58,346.320,2019-02-16,253902,2019-02-15 08:27:12,5297,-1,1,230,64


In [28]:
p.drop_duplicates(subset=None, keep='first', inplace=True)
p.head()

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm,广告请求时间_date,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
0,53991770,2019-02-17 13:22:26,94,1160618,451525,50,46,47.217,944.34,3122.340,2019-02-17,451525,2019-02-15 10:11:02,25082,-1,2,172,50
1,25942318,2019-02-17 02:34:52,79,203814,214797,64,10,49.094,981.88,1471.880,2019-02-17,214797,2019-02-13 02:57:43,22770,342,18,232,64
2,66156247,2019-02-17 15:16:40,18,808543,92253,40,96,3.824,76.48,443.584,2019-02-17,92253,2018-10-18 07:10:57,29218,32847,13,172,40
3,5935886,2019-02-17 01:11:38,198,7270,160082,64,85,6.123,122.46,642.915,2019-02-17,160082,2019-02-16 08:29:28,12759,-1,1,90,64
4,11624425,2019-02-16 23:52:39,168,852707,253902,64,60,4.329,86.58,346.320,2019-02-16,253902,2019-02-15 08:27:12,5297,-1,1,230,64


In [29]:
p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 48
Data columns (total 18 columns):
广告请求id              48 non-null int64
广告请求时间              48 non-null datetime64[ns]
广告位id               48 non-null int64
用户id                48 non-null int64
曝光广告id              48 non-null int64
曝光广告素材尺寸            48 non-null int64
曝光广告出价bid           48 non-null int64
曝光广告pctr            48 non-null float64
曝光广告quality_ecpm    48 non-null float64
曝光广告totalEcpm       48 non-null float64
广告请求时间_date         48 non-null object
广告id                48 non-null int64
创建时间                48 non-null object
广告账户id              48 non-null int64
商品id                48 non-null int64
商品类型                48 non-null int64
广告行业id              48 non-null int64
素材尺寸                48 non-null object
dtypes: datetime64[ns](1), float64(3), int64(11), object(3)
memory usage: 7.1+ KB


- 逐行保存为 csv 测试

In [30]:
# p.apply(save_csv, axis=1);

In [31]:
col_names1 = list(p.columns)
col_names1

['广告请求id',
 '广告请求时间',
 '广告位id',
 '用户id',
 '曝光广告id',
 '曝光广告素材尺寸',
 '曝光广告出价bid',
 '曝光广告pctr',
 '曝光广告quality_ecpm',
 '曝光广告totalEcpm',
 '广告请求时间_date',
 '广告id',
 '创建时间',
 '广告账户id',
 '商品id',
 '商品类型',
 '广告行业id',
 '素材尺寸']

In [32]:
logs1 = pd.read_csv(log_data/'totalExposureLog.out', sep='\t', header=None, names=col_names, chunksize=c_sz)

- 这里删除已经存在的文件

In [33]:
purge_pat_files('../data', r'^[^_]+_log.csv$')

In [ ]:
col = ['广告请求id',
 '广告请求时间',
 '广告位id',
 '用户id',
 '曝光广告id']

In [ ]:
for df, _ in zip(logs1, trange(1000)):
    df = pd.merge(df, ad_static, left_on='曝光广告id', right_on='广告id', how='inner')
    # 3. 去掉非法时间行
    df['广告请求时间'] = pd.to_datetime(df['广告请求时间'], unit='s')  # 转为日期
    df = df.apply(invalid_date, axis=1)
    
    # 1. 去空值
    df.dropna(axis=0, how='any', inplace=True)
    df = df[col]
    # 2. 去重 所有列相同
    df.drop_duplicates(subset=None, keep='first', inplace=True)
    # 4. 数据类型转换
    # 暂无
    # 数据分割
    df['广告请求时间_date'] = df['广告请求时间'].apply(lambda x: x.date())
    _ = df.apply(save_csv, axis=1)
#     df.to_csv('../data/ad_logs.csv', mode='a', index=None, encoding='utf-8', header=False)

In [36]:
col_names1 = ['广告请求id', '广告请求时间', '广告位id', '用户id', '曝光广告id', '广告请求时间_date']

- 重新读入 全部数据

In [ ]:
def read_files(dir, pattern):
    fx = os.listdir(dir)
    for f, _ in zip(fx, trange(len(fx))):
        if re.search(pattern, f):
            data = pd.read_csv(f'../data/{f}', sep='\t', header=None, names=col_names1)
            data.drop_duplicates(subset=None, keep='first', inplace=True)
            data.to_csv(f'../data/{f}', mode='w', index=None, encoding='utf-8')

 - 再次去重

In [ ]:
read_files('../data', r'^[^_]+_log.csv$')